# Lab: DataFrame API and Spark SQL


## Social characteristics of the Marvel Universe

In this lab, you be working with a dataset that was created about the Marvel Comics universe. The source data is a text file that was created with a graph analysis library outside of Spark, so the text file contains a lot of information and is not in a format that is easy to query with SQL. Do not worry that it mentions the word "graph", you can think of these tables like regular SQL tables. You are going to use the file in this lab to practice data ingestion, manipulation, and analysis using both the DataFrame API and Spark SQL. You can see more about the source data [here](http://bioinfo.uib.es/~joemiro/marvel.html).

## Understanding the data file

As mentioned above, the data file contains three different tables worth of info:

- data on Marvel characters
- data on Marvel publications
- data on which character appeared in which publication

This data is not yet separated into three different tables. It is all combined into a single file. You will use PySparkSQL to extract the information needed. Each character has its own **primary key**, and each publication has its own **primary key**. The third table will have the connections between character **primary keys** and publication **primary keys**. 

Eventually you will combine all three tables together into an analytics dataset, where you will answer the following questions:

- What are the top 10 publications with the most characters in them?
- What are the top 10 characters who appear in the most publications?

### The details of the data file are critical to extracting the data properly. The file is separated into two sections:

#### The characters/publications (aka vertices or nodes) (section begins in line 1 with a header in the form of `*Vertices 19428 6486`):

- Characters: lines 2-6487 in the format `1 "24-HOUR MAN/EMMANUEL"`, where `1` is the character primary key (aka node id) and the name of the character is within double quotes. The character table should look like the table sample below after processing:

|character_id|           character|
|------------|--------------------|
|           1|24-HOUR MAN/EMMANUEL|
|           2|3-D MAN/CHARLES CHAN|

- Publications: lines 6488-19429 in the same format as characters like `6487 "AA2 35"` with the publication primary key and the name of the publication. The publciation table should look like the table sample below after processing.

|publication_id|publication|
|--------------|-----------|
|          6487|     AA2 35|
|          6488|   M/PRM 35|
|          6489|   M/PRM 36|
|          6490|   M/PRM 37|
|          6491|      WI? 9|
|          6492|      AVF 4|
|          6493|      AVF 5|
|          6494|     H2 251|
|          6495|     H2 252|
|          6496|      COC 1|


#### The relationships (aka edges list) (section begins in line 19430 with a header in the form of `*Edgeslist`) lines 19431 to the end of the file. 

- The relationships information is in the following format: `2 6487 6488 6489 6490 6491 6492 6493 6494 6495 6496`. This represents a relationship between the character primary key (the first number) and the publication primary keys (all other numbers.). The relationships table should look like the following table sample after processing.

|character_id|publication_id|
|------------|--------------|
|           1|          6487|
|           2|          6488|
|           2|          6489|
|           2|          6490|
|           2|          6491|
|           2|          6492|
|           2|          6493|
|           2|          6494|
|           2|          6495|
|           2|          6496|

#### After combining the relationships with the character and publication information, the analytics table table should look like the table sample below:

|character_id|           character|publication_id|publication|
|------------|--------------------|--------------|-----------|
|           1|24-HOUR MAN/EMMANUEL|          6487|     AA2 35|
|           2|3-D MAN/CHARLES CHAN|          6488|   M/PRM 35|
|           2|3-D MAN/CHARLES CHAN|          6489|   M/PRM 36|
|           2|3-D MAN/CHARLES CHAN|          6490|   M/PRM 37|
|           2|3-D MAN/CHARLES CHAN|          6491|      WI? 9|
|           2|3-D MAN/CHARLES CHAN|          6492|      AVF 4|
|           2|3-D MAN/CHARLES CHAN|          6493|      AVF 5|
|           2|3-D MAN/CHARLES CHAN|          6494|     H2 251|
|           2|3-D MAN/CHARLES CHAN|          6495|     H2 252|
|           2|3-D MAN/CHARLES CHAN|          6496|      COC 1|

#### The goal is to clean the data tables so that you can easily leverage SQL commands to join the tables together

## Let's get started!

Find all your Spark related environment variables, and pyspark using the `findspark.init()` function:

In [1]:
import findspark
findspark.init()

Create your SparkSession. You are only going to create a `SparkSession`, not a `SparkContext`.

In [2]:
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.appName("my-app-name").getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/14 23:48:11 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
22/10/14 23:48:19 WARN JettyUtils: GET /jobs/ failed: org.apache.spark.SparkException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
org.apache.spark.SparkException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
	at org.apache.spark.status.AppStatusStore.applicationInfo(AppStatusStore.scala:44)
	at org.apache.spark.ui.jobs.AllJobsPage.render(AllJobsPage.scala:266)
	at org.apache.spark.ui.WebUI.$anonfun$attachPage$1(WebUI.scala:89)
	at org.apache.spark.ui.JettyUtils$$anon$1.doGet(JettyUtils.scala:80)
	at javax.servlet.http.HttpServlet.service(HttpServlet.java:687)
	at javax.servlet.http.HttpSer

Make sure your SparkSession is active by running the `spark` line:

In [3]:
spark

## Read in the data.

Although the data is a flat file with the structure explained earlier, you will be working with this file using [Spark DataFrame API and Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html).

Load in the text file using [Generic load functions for SparkSQL](https://spark.apache.org/docs/latest/sql-programming-guide.html#data-sources), which is located at `s3://bigdatateaching/marvel/porgat.txt`. This file is also located in Azure Blob at `wasbs://marvel@bigdatateaching.blob.core.windows.net/porgat.txt`. 

Create a DataFrame called `df_in`, which will have a single field, where each record is the full text per line in the original text file.


In [4]:
df_in = spark.read.text('s3://bigdatateaching/marvel/porgat.txt')

Look at the first few lines of `df_in`. Using the `.show()` method. What is the default field name?

In [5]:
df_in.show()

+--------------------+
|               value|
+--------------------+
|*Vertices 19428 6486|
|1 "24-HOUR MAN/EM...|
|2 "3-D MAN/CHARLE...|
|3 "4-D MAN/MERCURIO"|
|         4 "8-BALL/"|
|               5 "A"|
|           6 "A'YIN"|
|    7 "ABBOTT, JACK"|
|         8 "ABCISSA"|
|            9 "ABEL"|
|10 "ABOMINATION/E...|
|11 "ABOMINATION |...|
|    12 "ABOMINATRIX"|
|        13 "ABRAXAS"|
|     14 "ADAM 3,031"|
|        15 "ABSALOM"|
|16 "ABSORBING MAN...|
|17 "ABSORBING MAN...|
|           18 "ACBA"|
|19 "ACHEBE, REVER...|
+--------------------+
only showing top 20 rows



 Count the number of rows in `df_in` using the `count()` method:

In [6]:
df_in.count()

30520

Check the schema of your dataframe `df_in` using the method `.printSchema()`:

In [7]:
df_in.printSchema()

root
 |-- value: string (nullable = true)



Consider where the data is going to end up when you run the `take` command on a DataFrame. Get the first few records of the `df_in` and save it into an object in your Python session:

In [8]:
df_in_rec = df_in.take(10)

Explore the local Python object. How is it different than the DataFrame in the cluster? Read up on the [Pyspark Row object](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=row#pyspark.sql.Row)

In [9]:
df_in_rec

[Row(value='*Vertices 19428 6486'),
 Row(value='1 "24-HOUR MAN/EMMANUEL"'),
 Row(value='2 "3-D MAN/CHARLES CHAN"'),
 Row(value='3 "4-D MAN/MERCURIO"'),
 Row(value='4 "8-BALL/"'),
 Row(value='5 "A"'),
 Row(value='6 "A\'YIN"'),
 Row(value='7 "ABBOTT, JACK"'),
 Row(value='8 "ABCISSA"'),
 Row(value='9 "ABEL"')]

It is critical not to "take" too much data from your cluster to your local session. There is no hard and fast rule, but more than 10,000 rows could start to slow down your system.

## Arrange the data

Since the data is in text format, at the end of this section you will have three DataFrames: one for the characters, one for the publications, and one for the relationship between characters and publications.



The DataFrame API and SparkSQL functions are in the [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions) library. You can import all of the functions or specific functions as needed.

Start by importing the `monotonically_increasing_id` function from the `pyspark.sql.functions` library:

In [10]:
from pyspark.sql.functions import monotonically_increasing_id

Create a new dataframe called `df_in_idx` where you will add a new field called `idx` that uses the `monotonically_increasing_id` to add a unique incremental numeric index to each record that increases from 1 at the beginning of the data to N at the end of the data. You should read more about the function and understand how it works [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.monotonically_increasing_id.html). 

To make a new column, you must use the method `.withColumn([NEW COL NAME], [CODE TO MAKE COL])`. Read more about the method [here](https://sparkbyexamples.com/pyspark/pyspark-withcolumn/).

In [11]:
df_in_idx = df_in.withColumn('idx',monotonically_increasing_id())

Show your new dataframe, you will see the new column.

In [12]:
df_in_idx.show()

+--------------------+---+
|               value|idx|
+--------------------+---+
|*Vertices 19428 6486|  0|
|1 "24-HOUR MAN/EM...|  1|
|2 "3-D MAN/CHARLE...|  2|
|3 "4-D MAN/MERCURIO"|  3|
|         4 "8-BALL/"|  4|
|               5 "A"|  5|
|           6 "A'YIN"|  6|
|    7 "ABBOTT, JACK"|  7|
|         8 "ABCISSA"|  8|
|            9 "ABEL"|  9|
|10 "ABOMINATION/E...| 10|
|11 "ABOMINATION |...| 11|
|    12 "ABOMINATRIX"| 12|
|        13 "ABRAXAS"| 13|
|     14 "ADAM 3,031"| 14|
|        15 "ABSALOM"| 15|
|16 "ABSORBING MAN...| 16|
|17 "ABSORBING MAN...| 17|
|           18 "ACBA"| 18|
|19 "ACHEBE, REVER...| 19|
+--------------------+---+
only showing top 20 rows



We know from the data file that the two headers are in lines 1 and 19430, and we want those lines (records) from the data. Filter the data using a regex to only include values rows that start with a `*`. The code is provided in the cell below.

In [13]:
# a regex search on the value column for rows that start with *
df_in_idx.filter(df_in.value.rlike('^\*')).show()

+--------------------+---+
|               value|idx|
+--------------------+---+
|*Vertices 19428 6486|  0|
|          *Edgeslist|  1|
+--------------------+---+



Note: after we make any change to our dataframe, we need to examine the results. This is accomplished using the `.show()` method as shown above.

Now we want to create a new dataframe called `df_idx_no_hdr` where using a sql query where you select all records but those with the header. But wait...

Before you can run a SparkSQL query, you need to "register" a dataframe. This can be accomplished with the method `.createOrReplaceTempView([TABLE_NAME])`. Set up the "register" your dataframe `df_in_idx`

In [14]:
df_in_idx.createOrReplaceTempView('my_table')

Now you can use the function `spark.sql([SQL STATEMENT])` to make the new dataframe `df_idx_no_hdr` that removes the rows with header info.

In [15]:
df_idx_no_hdr = spark.sql("select * from my_table where idx !=0 and idx!=19429")

Check that the header rows were removed by counting the number of rows in the resulting dataframe:

In [16]:
df_idx_no_hdr.count()

30518

In [17]:
df_idx_no_hdr.take(10)

[Row(value='1 "24-HOUR MAN/EMMANUEL"', idx=1),
 Row(value='2 "3-D MAN/CHARLES CHAN"', idx=2),
 Row(value='3 "4-D MAN/MERCURIO"', idx=3),
 Row(value='4 "8-BALL/"', idx=4),
 Row(value='5 "A"', idx=5),
 Row(value='6 "A\'YIN"', idx=6),
 Row(value='7 "ABBOTT, JACK"', idx=7),
 Row(value='8 "ABCISSA"', idx=8),
 Row(value='9 "ABEL"', idx=9),
 Row(value='10 "ABOMINATION/EMIL BLO"', idx=10)]

## Creating Tables

Now you will create three separate dataframes: a `characters` one, a `publications` one, and a `relationships` one. You know the original line indices that partition the file, so use those. You have the `idx` field to use.

You will need to subset the dataframe `df_idx_no_hdr` using the `.filter()` method. The `col([VAR NAME])` function is used within a dataframe method to refer to a variable using the variable name. After making your dataframes, check the number of rows and show the first few rows for each.

In [18]:
from pyspark.sql.functions import col

In [19]:
#df_idx_no_hdr.select("value", "idx").write.save("nohdr.csv")

In [20]:
df_idx_no_hdr.show()

+--------------------+---+
|               value|idx|
+--------------------+---+
|1 "24-HOUR MAN/EM...|  1|
|2 "3-D MAN/CHARLE...|  2|
|3 "4-D MAN/MERCURIO"|  3|
|         4 "8-BALL/"|  4|
|               5 "A"|  5|
|           6 "A'YIN"|  6|
|    7 "ABBOTT, JACK"|  7|
|         8 "ABCISSA"|  8|
|            9 "ABEL"|  9|
|10 "ABOMINATION/E...| 10|
|11 "ABOMINATION |...| 11|
|    12 "ABOMINATRIX"| 12|
|        13 "ABRAXAS"| 13|
|     14 "ADAM 3,031"| 14|
|        15 "ABSALOM"| 15|
|16 "ABSORBING MAN...| 16|
|17 "ABSORBING MAN...| 17|
|           18 "ACBA"| 18|
|19 "ACHEBE, REVER...| 19|
|       20 "ACHILLES"| 20|
+--------------------+---+
only showing top 20 rows



In [21]:
characters = df_idx_no_hdr.filter((col('idx')<=6486))

In [22]:
characters.count()

6486

In [23]:
characters.take(10)

[Row(value='1 "24-HOUR MAN/EMMANUEL"', idx=1),
 Row(value='2 "3-D MAN/CHARLES CHAN"', idx=2),
 Row(value='3 "4-D MAN/MERCURIO"', idx=3),
 Row(value='4 "8-BALL/"', idx=4),
 Row(value='5 "A"', idx=5),
 Row(value='6 "A\'YIN"', idx=6),
 Row(value='7 "ABBOTT, JACK"', idx=7),
 Row(value='8 "ABCISSA"', idx=8),
 Row(value='9 "ABEL"', idx=9),
 Row(value='10 "ABOMINATION/EMIL BLO"', idx=10)]

In [24]:
publications = df_idx_no_hdr.filter((col('idx')>6486)&(col('idx')<=19428))

In [25]:
publications.count()

12942

In [26]:
publications.take(10)

[Row(value='6487 "AA2 35"', idx=6487),
 Row(value='6488 "M/PRM 35"', idx=6488),
 Row(value='6489 "M/PRM 36"', idx=6489),
 Row(value='6490 "M/PRM 37"', idx=6490),
 Row(value='6491 "WI? 9"', idx=6491),
 Row(value='6492 "AVF 4"', idx=6492),
 Row(value='6493 "AVF 5"', idx=6493),
 Row(value='6494 "H2 251"', idx=6494),
 Row(value='6495 "H2 252"', idx=6495),
 Row(value='6496 "COC 1"', idx=6496)]

In [27]:
relationships = df_idx_no_hdr.filter(col('idx')>19428)

In [28]:
relationships.count()

11090

In [29]:
relationships.take(10)

[Row(value='1 6487', idx=19430),
 Row(value='2 6488 6489 6490 6491 6492 6493 6494 6495 6496', idx=19431),
 Row(value='3 6497 6498 6499 6500 6501 6502 6503 6504 6505', idx=19432),
 Row(value='4 6506 6507 6508', idx=19433),
 Row(value='5 6509 6510 6511', idx=19434),
 Row(value='6 6512 6513 6514 6515', idx=19435),
 Row(value='7 6516', idx=19436),
 Row(value='8 6517 6518', idx=19437),
 Row(value='9 6519 6520', idx=19438),
 Row(value='10 6521 6522 6523 6524 6525 6526 6527 6528 6529 6530 6531 6532 6533 6534 6535', idx=19439)]

## Extracting Value from Data

Register the characters and publications dataframes in order to run SQL commands:

In [30]:
characters.createOrReplaceTempView('cha_table')

In [31]:
publications.createOrReplaceTempView('pub_table')

Use the `regexp_extract` function within a SQL statement and a regular expression to create three fields from the whole line in the `character` and `publication` tables: 

1. the first field will be the integer (which is the node id)
2. the second is the text **between** the double quotes
3. the third is a string variable with the value "character" or "publication". 

For help with the regex, try going to https://regex101.com/ and paste in some of the lines to test the regex query.

Create two new separate dataframes, one for characters and the other for publications.

In [32]:
char_df = spark.sql("SELECT idx as node_id, regexp_extract(value, '\"(.*)\"',1) as value FROM cha_table")

In [33]:
from pyspark.sql.functions import lit
char_df = char_df.withColumn("label", lit("character"))

In [34]:
publication_df = spark.sql("SELECT idx as node_id, regexp_extract(value, '\"(.*)\"',1) as value FROM pub_table")

In [35]:
publication_df = publication_df.withColumn("label", lit("publication"))

The results should look like the following two cells:

In [36]:
char_df.show(5)

+-------+--------------------+---------+
|node_id|               value|    label|
+-------+--------------------+---------+
|      1|24-HOUR MAN/EMMANUEL|character|
|      2|3-D MAN/CHARLES CHAN|character|
|      3|    4-D MAN/MERCURIO|character|
|      4|             8-BALL/|character|
|      5|                   A|character|
+-------+--------------------+---------+
only showing top 5 rows



In [37]:
publication_df.show(5)

+-------+--------+-----------+
|node_id|   value|      label|
+-------+--------+-----------+
|   6487|  AA2 35|publication|
|   6488|M/PRM 35|publication|
|   6489|M/PRM 36|publication|
|   6490|M/PRM 37|publication|
|   6491|   WI? 9|publication|
+-------+--------+-----------+
only showing top 5 rows



Stack both the `character` and `publication` dataframes into a single dataframe called `nodes_df`, and cache it.

Stacking dataframes by rows can be accomplished using the `.union()` method, which you can read about [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.union.html).

In [38]:
nodes_df = char_df.union(publication_df)

In [39]:
nodes_df.cache()

DataFrame[node_id: bigint, value: string, label: string]

Now, you will work on the `relationships` dataframe. Import the `split` and `explode` functions from the `pyspark.sql.functions` library. Review the following cells in preparation for the major chunk of code you will write. These cells are to show you examples of running pyspark commands with the split method.

In [40]:
from pyspark.sql.functions import split, explode

In [41]:
relationships.show()

+--------------------+-----+
|               value|  idx|
+--------------------+-----+
|              1 6487|19430|
|2 6488 6489 6490 ...|19431|
|3 6497 6498 6499 ...|19432|
|    4 6506 6507 6508|19433|
|    5 6509 6510 6511|19434|
|6 6512 6513 6514 ...|19435|
|              7 6516|19436|
|         8 6517 6518|19437|
|         9 6519 6520|19438|
|10 6521 6522 6523...|19439|
|10 6536 6537 6538...|19440|
|10 6551 6552 6553...|19441|
|             11 6566|19442|
|   12 6567 6568 6569|19443|
|13 6570 6571 6572...|19444|
|14 6574 6575 6576...|19445|
|15 6578 6579 6580...|19446|
|16 6582 6583 6584...|19447|
|16 6597 6598 6599...|19448|
|16 6612 6613 6614...|19449|
+--------------------+-----+
only showing top 20 rows



In [42]:
r2 = relationships.withColumn("newcol", split(col("value"), " ")).cache() #convert a single text to array

In [43]:
r2.show()

+--------------------+-----+--------------------+
|               value|  idx|              newcol|
+--------------------+-----+--------------------+
|              1 6487|19430|           [1, 6487]|
|2 6488 6489 6490 ...|19431|[2, 6488, 6489, 6...|
|3 6497 6498 6499 ...|19432|[3, 6497, 6498, 6...|
|    4 6506 6507 6508|19433|[4, 6506, 6507, 6...|
|    5 6509 6510 6511|19434|[5, 6509, 6510, 6...|
|6 6512 6513 6514 ...|19435|[6, 6512, 6513, 6...|
|              7 6516|19436|           [7, 6516]|
|         8 6517 6518|19437|     [8, 6517, 6518]|
|         9 6519 6520|19438|     [9, 6519, 6520]|
|10 6521 6522 6523...|19439|[10, 6521, 6522, ...|
|10 6536 6537 6538...|19440|[10, 6536, 6537, ...|
|10 6551 6552 6553...|19441|[10, 6551, 6552, ...|
|             11 6566|19442|          [11, 6566]|
|   12 6567 6568 6569|19443|[12, 6567, 6568, ...|
|13 6570 6571 6572...|19444|[13, 6570, 6571, ...|
|14 6574 6575 6576...|19445|[14, 6574, 6575, ...|
|15 6578 6579 6580...|19446|[15, 6578, 6579, ...|


Now, using a chain of dataframe api methods, start with the value field in the `relationships` dataframe, where you will create a dataframe that takes the first value of the edge and puts it in the character field, and then "explodes" the other values in the publication column (1 row per character-publication combination).

To make a new column, you must use the method `.withColumn([NEW COL NAME], [CODE TO MAKE COL])`. Read more about the method [here](https://sparkbyexamples.com/pyspark/pyspark-withcolumn/).

There are six chained methods necessary:

1. make a new column using the `split` function from column `value` as we just did above in the example code
2. make a new column `character` by grabbing the first item from the list column you just made in step 1. Read about a [useful list column function](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.getItem.html).
3. make a new column `publication` by using the `explode` function to make each publication in a list as a separate row. Read more about the `explode` function [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.explode.html).
4. Select only the `character` and `publication` columns
5. Filter the dataframe to include only rows where the character and publication columns are not equal
6. Cache your results
7. Save the final dataframe as `edges_df`

After each step 1-5 you should use a `.show(10, truncate = False)` method to examine the current state of your dataframe.


In [44]:
r_1 = relationships.withColumn("newcol", split(col("value"), " "))
r_1.show(10, truncate = False)

+-----------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------+
|value                                                                        |idx  |newcol                                                                                        |
+-----------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------+
|1 6487                                                                       |19430|[1, 6487]                                                                                     |
|2 6488 6489 6490 6491 6492 6493 6494 6495 6496                               |19431|[2, 6488, 6489, 6490, 6491, 6492, 6493, 6494, 6495, 6496]                                     |
|3 6497 6498 6499 6500 6501 6502 6503 6504 6505                               |19432|[3, 6497, 

In [45]:
r_2 = r_1.withColumn("character", col('newcol').getItem(0))
r_2.show(10, truncate = False)

+-----------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------+---------+
|value                                                                        |idx  |newcol                                                                                        |character|
+-----------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------+---------+
|1 6487                                                                       |19430|[1, 6487]                                                                                     |1        |
|2 6488 6489 6490 6491 6492 6493 6494 6495 6496                               |19431|[2, 6488, 6489, 6490, 6491, 6492, 6493, 6494, 6495, 6496]                                     |2        |
|3 6497 6498 6499 6500 6501 6502 6503 6504 65

In [46]:
r_3 = r_2.withColumn("publication", explode(split(col("value"), " ")))
r_3.show(10, truncate = False)

+----------------------------------------------+-----+---------------------------------------------------------+---------+-----------+
|value                                         |idx  |newcol                                                   |character|publication|
+----------------------------------------------+-----+---------------------------------------------------------+---------+-----------+
|1 6487                                        |19430|[1, 6487]                                                |1        |1          |
|1 6487                                        |19430|[1, 6487]                                                |1        |6487       |
|2 6488 6489 6490 6491 6492 6493 6494 6495 6496|19431|[2, 6488, 6489, 6490, 6491, 6492, 6493, 6494, 6495, 6496]|2        |2          |
|2 6488 6489 6490 6491 6492 6493 6494 6495 6496|19431|[2, 6488, 6489, 6490, 6491, 6492, 6493, 6494, 6495, 6496]|2        |6488       |
|2 6488 6489 6490 6491 6492 6493 6494 6495 6496|19431|[

In [47]:
r_4 = r_3.select(r_3.character, r_3.publication)
r_4.show(10, truncate = False)

+---------+-----------+
|character|publication|
+---------+-----------+
|1        |1          |
|1        |6487       |
|2        |2          |
|2        |6488       |
|2        |6489       |
|2        |6490       |
|2        |6491       |
|2        |6492       |
|2        |6493       |
|2        |6494       |
+---------+-----------+
only showing top 10 rows



In [48]:
edges_df = r_4.filter(r_4.character != r_4.publication) 
edges_df.show(10, truncate = False)

+---------+-----------+
|character|publication|
+---------+-----------+
|1        |6487       |
|2        |6488       |
|2        |6489       |
|2        |6490       |
|2        |6491       |
|2        |6492       |
|2        |6493       |
|2        |6494       |
|2        |6495       |
|2        |6496       |
+---------+-----------+
only showing top 10 rows



In [49]:
edges_df.cache()

DataFrame[character: string, publication: string]

Register your `nodes_df` and `edges_df` as SQL views

In [50]:
nodes_df.createOrReplaceTempView("nodes")
edges_df.createOrReplaceTempView("edges")

## Create an analytical dataset

You will now create an analytical dataset using SparkSQL where you will join both tables (nodes_df and edge_df) so you have the data you need to run some analytics on the data.

1. Start off by showing 10 rows from both datasets and get a count of rows. You can run the show and count commands in the same cell.
2. Then print the schemas of both dataframes to confirm data type match.

In [51]:
nodes_df.show(10)
nodes_df.count()

+-------+--------------------+---------+
|node_id|               value|    label|
+-------+--------------------+---------+
|      1|24-HOUR MAN/EMMANUEL|character|
|      2|3-D MAN/CHARLES CHAN|character|
|      3|    4-D MAN/MERCURIO|character|
|      4|             8-BALL/|character|
|      5|                   A|character|
|      6|               A'YIN|character|
|      7|        ABBOTT, JACK|character|
|      8|             ABCISSA|character|
|      9|                ABEL|character|
|     10|ABOMINATION/EMIL BLO|character|
+-------+--------------------+---------+
only showing top 10 rows



19428

In [52]:
edges_df.show(10)
edges_df.count()

+---------+-----------+
|character|publication|
+---------+-----------+
|        1|       6487|
|        2|       6488|
|        2|       6489|
|        2|       6490|
|        2|       6491|
|        2|       6492|
|        2|       6493|
|        2|       6494|
|        2|       6495|
|        2|       6496|
+---------+-----------+
only showing top 10 rows



96662

In [53]:
nodes_df.printSchema()

root
 |-- node_id: long (nullable = false)
 |-- value: string (nullable = true)
 |-- label: string (nullable = false)



In [54]:
edges_df.printSchema()

root
 |-- character: string (nullable = true)
 |-- publication: string (nullable = true)



There is an issue with the data types! We need to cast the `edges_df` columns to long format. This can be accomplished by calling the cast method on a variable of a dataframe, such as `df.var1.cast('str')`. See more examples of type [casting in PySpark here](https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/). You can also use the `col()` function we used previously like `col('var1').cast('str')`.

In [55]:
edges_df.character.cast('long')

Column<b'CAST(character AS BIGINT)'>

In [56]:
edges_df.publication.cast('long')

Column<b'CAST(publication AS BIGINT)'>

In [57]:
edges_df.printSchema()

root
 |-- character: string (nullable = true)
 |-- publication: string (nullable = true)



The edges dataframe needs information about each node. You will merge the nodes data for the publications and for the characters. This can be accomplished using Spark SQL or using Spark DataFrame commands.

Read about [joins in PySparkSQL here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.join.html). If you want to use PySparkSQL, you have to use the `char_df` and `publication_df` instead of nodes_df.

Make sure the node_id columns are renamed to `character_id` and `publication_id` using the method like `df.withColumnRenamed('var1','var2')`. Do the same for the name columns to `character` and `publication`.

Drop the node_type columns using the method `df.drop('var1')`.

The final dataset should look like:

|character_id|           character|publication_id|publication|
|------------|--------------------|--------------|-----------|
|           1|24-HOUR MAN/EMMANUEL|          6487|     AA2 35|
|           2|3-D MAN/CHARLES CHAN|          6488|   M/PRM 35|
|           2|3-D MAN/CHARLES CHAN|          6489|   M/PRM 36|
|           2|3-D MAN/CHARLES CHAN|          6490|   M/PRM 37|
|           2|3-D MAN/CHARLES CHAN|          6491|      WI? 9|
|           2|3-D MAN/CHARLES CHAN|          6492|      AVF 4|
|           2|3-D MAN/CHARLES CHAN|          6493|      AVF 5|
|           2|3-D MAN/CHARLES CHAN|          6494|     H2 251|
|           2|3-D MAN/CHARLES CHAN|          6495|     H2 252|
|           2|3-D MAN/CHARLES CHAN|          6496|      COC 1|

In [58]:
edges_df = edges_df.withColumnRenamed('character','character_id')
edges_df = edges_df.withColumnRenamed('publication','publication_id')
edges_df.show(5)

+------------+--------------+
|character_id|publication_id|
+------------+--------------+
|           1|          6487|
|           2|          6488|
|           2|          6489|
|           2|          6490|
|           2|          6491|
+------------+--------------+
only showing top 5 rows



In [59]:
char_df.show(5)

+-------+--------------------+---------+
|node_id|               value|    label|
+-------+--------------------+---------+
|      1|24-HOUR MAN/EMMANUEL|character|
|      2|3-D MAN/CHARLES CHAN|character|
|      3|    4-D MAN/MERCURIO|character|
|      4|             8-BALL/|character|
|      5|                   A|character|
+-------+--------------------+---------+
only showing top 5 rows



In [64]:
char_df = char_df.withColumnRenamed('node_id','character_id').drop('label')

In [65]:
char_df = char_df.withColumnRenamed('value','character')

In [66]:
char_df.show(5)

+------------+--------------------+
|character_id|           character|
+------------+--------------------+
|           1|24-HOUR MAN/EMMANUEL|
|           2|3-D MAN/CHARLES CHAN|
|           3|    4-D MAN/MERCURIO|
|           4|             8-BALL/|
|           5|                   A|
+------------+--------------------+
only showing top 5 rows



In [62]:
publication_df.show(5)

+-------+--------+-----------+
|node_id|   value|      label|
+-------+--------+-----------+
|   6487|  AA2 35|publication|
|   6488|M/PRM 35|publication|
|   6489|M/PRM 36|publication|
|   6490|M/PRM 37|publication|
|   6491|   WI? 9|publication|
+-------+--------+-----------+
only showing top 5 rows



In [63]:
publication_df = publication_df.withColumnRenamed('node_id','publication_id').drop('label')

In [67]:
publication_df = publication_df.withColumnRenamed('value','publication')

In [68]:
publication_df.show(5)

+--------------+-----------+
|publication_id|publication|
+--------------+-----------+
|          6487|     AA2 35|
|          6488|   M/PRM 35|
|          6489|   M/PRM 36|
|          6490|   M/PRM 37|
|          6491|      WI? 9|
+--------------+-----------+
only showing top 5 rows



In [69]:
edges_df.createOrReplaceTempView('edges')
char_df.createOrReplaceTempView('char')
publication_df.createOrReplaceTempView('pub')

In [71]:
final_df = spark.sql("SELECT character_id,character,publication_id,publication FROM edges LEFT JOIN char using(character_id) LEFT JOIN pub using (publication_id)")

In [72]:
final_df.show(10)

+------------+--------------------+--------------+-----------+
|character_id|           character|publication_id|publication|
+------------+--------------------+--------------+-----------+
|           1|24-HOUR MAN/EMMANUEL|          6487|     AA2 35|
|           2|3-D MAN/CHARLES CHAN|          6488|   M/PRM 35|
|           2|3-D MAN/CHARLES CHAN|          6489|   M/PRM 36|
|           2|3-D MAN/CHARLES CHAN|          6490|   M/PRM 37|
|           2|3-D MAN/CHARLES CHAN|          6491|      WI? 9|
|           2|3-D MAN/CHARLES CHAN|          6492|      AVF 4|
|           2|3-D MAN/CHARLES CHAN|          6493|      AVF 5|
|           2|3-D MAN/CHARLES CHAN|          6494|     H2 251|
|           2|3-D MAN/CHARLES CHAN|          6495|     H2 252|
|           2|3-D MAN/CHARLES CHAN|          6496|      COC 1|
+------------+--------------------+--------------+-----------+
only showing top 10 rows



# Conduct Analytics on the Dataset

What are the top 10 publications with the highest number of characters present in them? Use Spark SQL to answer this question. 

In [78]:
import pyspark.sql.functions as f
# once you experiment and write the proper code then save to the dictionary key below
dict_answers = {}
dict_answers['q1'] = (final_df
                      .groupby('publication')
                      .agg(  f.count(col('character_id')).alias('char_ct')  )
                      .sort(  col('char_ct').desc()  )
                      .take(10)
                     )

What are the top 10 characters who appear in the most publications? Use dataframe methods to answer this question. You will need to use [groupby](https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/), [agg](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.GroupedData.agg.html), and [sort](https://sparkbyexamples.com/pyspark/pyspark-orderby-and-sort-explained/) functions.

Example: 

```
import pyspark.sql.functions as f
(df
  .groupby('grouping_variable')
  .agg(  f.count(col('var1')).alias('var1_ct')  )
  .sort(  col('var1_ct').desc()  )
  .show(10)
)
```

In [79]:

# once you experiment and write the proper code then save to the dictionary key below
dict_answers['q2'] = (final_df
                      .groupby('character')
                      .agg(  f.count(col('publication_id')).alias('pub_ct')  )
                      .sort(  col('pub_ct').desc()  )
                      .take(10)
                     )

## **Save your analytics results to a json object - then add, commit, and push your notebook and json to GitHub!**

In [80]:
import json
json.dump(str(dict_answers), fp = open('lab-solution.json','w'))

# STOP YOUR CLUSTER!

In [81]:
spark.stop()